In [7]:
import pandas as pd
from multiprocessing import Pool
import zipfile
import requests
import io
import os
from functools import partial
import sys
import boto3
import pymongo
from ast import literal_eval
from pymongo import MongoClient
import pprint
import pdb

#sys.argv[1] = '010-mentions-translation.csv'
#sys.argv[2] = 'mentions-header.txt'
#sys.argv[3] = 'mention-json-cols.txt'


filename = '070-mentions-translation.csv'
filelist = list(filter(lambda x:x!='',open(filename,'r').read().split('\n')))
headers = open('mentions-header.txt','r').read().split()
collist = open('mentions-json-cols.txt','r').read().split()

In [9]:
def csvToPd(headers,collist,fileurl):

    z = zipfile.ZipFile(io.BytesIO(requests.get(fileurl).content))
    df = pd.read_csv(z.open(fileurl.split('/')[-1][:-4]),delimiter='\t',names=headers,index_col=False).fillna('')[collist]
    df['MentionDocTranslationInfo'].replace('','srclc:eng')
    df['MentionDocTranslationInfo']=df['MentionDocTranslationInfo'].str[6:9]
    df = df.drop_duplicates()

    return df

p = Pool(10)
res = p.map(partial(csvToPd,headers,collist),filelist)
res = pd.concat(res).reset_index().drop('index',axis=1)

js = res.groupby('GlobalEventID',as_index='GlobalEventID').apply(lambda x:x.to_json(orient='records'))
js = js.apply(lambda x: literal_eval(x[1:-1]))

In [10]:
js.to_json('070.json',orient='records')

In [6]:
js.to_json('010.json',orient='records')